In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

03fc8903-f7fa-4609-89b9-71ac610ba4c2


In [11]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Smith explains complex concepts very well, but sometimes the pace is a bit fast.'},
 {'professor': 'Dr. Emily Brown',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Dr. Brown is an excellent professor. Her lectures are clear, and she is always willing to help students.'},
 {'professor': 'Dr. Alan Thompson',
  'subject': 'Computer Science',
  'stars': 3,
  'review': 'Dr. Thompson is knowledgeable, but his teaching style is a bit disorganized. Office hours are helpful.'},
 {'professor': 'Dr. Susan Lee',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Dr. Lee is very strict and her exams are extremely difficult. The material is hard to understand.'},
 {'professor': 'Dr. Michael Johnson',
  'subject': 'History',
  'stars': 5,
  'review': "Dr. Johnson's passion for history makes his classes engaging and enjoyable. Highly recommended!"},
 {'professor': 'Dr. Karen White',
  'subject': 'English

In [15]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review":review["review"],
            "subject":review["subject"],
            "stars":review["stars"],
        }
    })

In [16]:
processed_data[0]

{'values': [-0.022121929,
  0.043393012,
  -0.047537442,
  -0.017853988,
  0.025470408,
  0.019596845,
  0.013325306,
  0.029340373,
  0.04438109,
  -0.028544422,
  0.01154814,
  -0.021476934,
  -0.02046141,
  0.005726039,
  0.050474226,
  -0.05299931,
  -0.05371292,
  -0.059174784,
  0.00088515156,
  0.015617093,
  -0.005993643,
  -0.012199996,
  0.023535427,
  0.011108995,
  -0.056430127,
  -0.0040380755,
  0.037958592,
  0.02903846,
  0.039138794,
  -0.002718925,
  0.06340156,
  -0.026197743,
  0.029175693,
  -0.030493129,
  -0.037684128,
  0.03664116,
  -0.0060142274,
  0.013037116,
  -0.0003471561,
  0.0017042599,
  0.013634079,
  0.03488458,
  -0.067628324,
  -0.013043978,
  0.09271448,
  0.0022334638,
  -0.009695497,
  -0.011760851,
  0.005262878,
  0.048909772,
  -0.06422495,
  0.043365564,
  0.000779225,
  0.019720353,
  -0.018018667,
  0.029724626,
  0.032332048,
  0.010045441,
  0.021861186,
  -0.019665461,
  0.08354733,
  -0.033210337,
  -0.020886833,
  -0.0056368373,
  -0.

In [17]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 10}

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 10}},
 'total_vector_count': 10}